# PIMA DATASET
already preprocessed the dataset.

In [6]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import Row
from pyspark.sql import SparkSession
import csv
from pyspark.sql.functions import monotonically_increasing_id
spark = SparkSession.builder.getOrCreate()

f = open("wiki_result.csv")
reader = csv.reader(f,delimiter='|')
ww = []
for w in reader:
    ww.append(w)
ww= map(lambda p: Row(label=int(p[0]), text=str(p[1])),ww)
data = spark.createDataFrame(ww)
data.show()
f.close

+-----+--------------------+
|label|                text|
+-----+--------------------+
|    0|asia ( ) earth 's...|
|    0|unilev ( ) dutch-...|
|    0|eurasia combin co...|
|    0|eric hoffer ( jul...|
|    0|shaman ( shah-men...|
|    0|the asian giant h...|
|    0|list asian pornog...|
|    0|georgia ( ; georg...|
|    0|calligraphi ( gre...|
|    0|hornet ( insect g...|
|    0|asia argento ( it...|
|    0|time american wee...|
|    0|the pacif war , s...|
|    0|the boundari cont...|
|    0|the demograph rus...|
|    0|thi list common s...|
|    0|A humid subtrop c...|
|    0|aed albopictu ( s...|
|    0|buddhism ( ) reli...|
|    0|thi list list wor...|
+-----+--------------------+
only showing top 20 rows



<function close>

## Vertorize the Data into feature

In [7]:
from pyspark.ml.feature import VectorAssembler
label = ["label"]
assembler = VectorAssembler(
    inputCols=[x for x in data.columns if x not in label],
    outputCol='features')
data = assembler.transform(data)

IllegalArgumentException: u'Data type StringType is not supported.'

##  Split the Data

In [5]:
splits = data.select("label", "features").randomSplit([0.8, 0.2], 1234)
train = splits[1]
test = splits[0]

## Use NaiveBayes method to build a model

In [6]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

nb = NaiveBayes()
model = nb.fit(train)
predictions = model.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.61648177496


## Use DecisionTree method to build a model

In [7]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

dt = DecisionTreeClassifier()

model = dt.fit(train)

predictions = model.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.698890649762


## Use RandomForest method to build a model

In [10]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

rf = RandomForestClassifier()

model = rf.fit(train)

predictions = model.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy of RandomForest= " + str(accuracy))

Test set accuracy of RandomForest= 0.765451664025
